In [1]:
import ast
import pandas as pd
import numpy as np
import scipy as sp
import scipy.sparse


In [2]:
import io
from contextlib import contextmanager


@contextmanager
def quote_brackets(filename):
    with open(filename) as f:
        yield io.StringIO(f.read().replace('[','"[').replace(']',']"'))

def combine_features(df):
    def str_cat_df(df,cols):
        series = (df[c] for c in cols)
        series = list(map(lambda x: x.apply(str), series))
        return series[0].str.cat(series[1:]).astype('category')
    
    author_features=['verified', 'activity', 'defaultprofile', 'userurl']
    tweet_features=['hashtag', 'tweeturl', 'mentions', 'media']
    
    df.insert(0,'author_feature',str_cat_df(df, author_features))
    df.insert(1,'tweet_feature',str_cat_df(df, tweet_features))
    
    df.drop(columns=author_features, inplace=True)
    df.drop(columns=tweet_features, inplace=True)

def normalize_level_column(colname):
    if colname.startswith('level'):
        return int(colname.removeprefix('level'))-1
    try:
        return int(colname)
    except ValueError:
        return colname
    
def normalized_csv(oldfile, newfile=None, numlevels=11):
    df = pd.read_csv(oldfile, dtype={'author_feature':'str','tweet_feature':'str'})
    if 'author_feature' not in df:
        combine_features(df)
    df.drop(columns=['author','tree','retweets'], inplace=True, errors='ignore')
    df.rename(mapper=normalize_level_column, axis=1, inplace=True)
    for l in range(numlevels):
        if l not in df:
            df[l]=0
    if newfile is not None:
        df.to_csv(newfile, index=False)
    return df
    


In [48]:
with quote_brackets('vegan_20210729_analysed.csv') as f:
    vegan_a=normalized_csv(f, 'vegan_analyzed.csv')
    
fpoe_a=normalized_csv('fpoeanalysed.csv','fpoe_analyzed.csv')
neos_a=normalized_csv('neos_20210729_analysed.csv','neos_analyzed.csv')

In [47]:
fpoe_g=normalized_csv('fpoe_generated.csv')
fpoe_p=normalized_csv('fpoe_with_params.csv')
neos_g=normalized_csv('results-trees-neos_20201110-2021-12-10T19:58:39.150768.csv')
neos_p=normalized_csv('neos_with_params.csv')

In [50]:
def by_feature(df):
    df=df.groupby(['author_feature', 'tweet_feature']).agg(['sum','mean',list])
    return df.dropna().swaplevel(axis=1).sort_index(axis=1)

In [49]:
fpoe_a=by_feature(fpoe_a)
neos_a=by_feature(neos_a)
fpoe_g=by_feature(fpoe_g)
fpoe_p=by_feature(fpoe_p)
neos_g=by_feature(neos_g)
neos_p=by_feature(neos_p)

In [216]:
def mean(a, b):
    return (a['mean'] - b['mean']).dropna().mean(axis=1)

def compare(a,b):
    m=mean(a,b)
    return {'mae':m.abs().mean(),
            'me':m.mean(),
            'mywasserstein_levels': observed_levels_distance(a,b, lambda u,v:wasserstein_2d(np.array([u]).transpose(),np.array([v]).transpose())).mean(),
            'mwasserstein_levels': mean_levels_distance(a,b, scipy.stats.wasserstein_distance).mean(),
            'menergy_levels': mean_levels_distance(a,b, scipy.stats.energy_distance).mean()
           }

def table(analyzed, without_params, with_params):
    return pd.DataFrame([compare(analyzed,without_params),compare(analyzed,with_params)],index=['without params', 'with params'])

mean(neos_p,neos_a).mean()
#by_feature(neos_a).loc[('0000','0001')]

-0.10922888427086508

In [217]:
table(fpoe_a,fpoe_g,fpoe_p)    

,mae,me,mywasserstein_levels,mwasserstein_levels,menergy_levels
without params,0.553913,0.514472,0.587123,0.393364,0.349402
with params,0.268906,0.137954,0.882816,0.486923,0.493684


In [166]:
table(neos_a, neos_g, neos_p)    

,mae,me,mwasserstein_levels,menergy_levels
without params,0.437720,0.330634,0.379652,0.360213
with params,0.247649,0.109229,0.477714,0.522775


In [253]:
import scipy.stats

def observed_levels(df):
    df=df['sum']
    return df.apply(lambda x: [l for c in df.columns for l in [c]*x[c]], axis=1)

def observed_levels_distance(a, b, distance=scipy.stats.wasserstein_distance):
    df=pd.DataFrame({'a':observed_levels(a), 'b':observed_levels(b)}).dropna()
    return df.apply(lambda x:distance(*x),axis=1)

def mean_levels_distance(a, b, distance=scipy.stats.wasserstein_distance):
    df=pd.concat([a['mean'],b['mean']],keys=['a','b'],axis=1).dropna()
    columns=a['mean'].columns
    return df.apply(lambda x: distance(columns,columns,x['a'],x['b']),axis=1)

mean_levels_distance(fpoe_a,fpoe_p).mean()
observed_levels_distance(fpoe_a,fpoe_p).mean()
#len(observed_levels(fpoe_a)[('0000','0000')])

0.48692311212143624

In [258]:
# https://stackoverflow.com/a/57563383/
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

def wasserstein_2d(U,V, normalize=True):
    if normalize:
        U = U#/U.shape[0]#U.sum().sum()
        V = V#/V.shape[0]#V.sum().sum()
    d = cdist(U,V)
    display(d)

    assignment = linear_sum_assignment(d)
    return d[assignment].sum()#/(U.sum().sum()+V.sum().sum())

def wasserstein_2d_row_ab(row, normalize=False):
    a=np.array(row['a'].to_list()).transpose()
    b=np.array(row['b'].to_list()).transpose()
    return wasserstein_2d(a,b,normalize)

def wasserstein_2d_df(a,b,normalize=False):
    a=a['list']
    b=b['list']
#     a=pd.Series([observed_levels(a)])
#     b=pd.Series([observed_levels(b)])
    df=pd.concat([a,b], keys=['a','b'], axis=1).dropna()
#     display(df)
    return df.apply(wasserstein_2d_row_ab, axis=1, normalize=normalize)

# display(observed_levels_distance(fpoe_a,fpoe_p))

wasserstein_2d_df(fpoe_a,fpoe_g,True)
# mine=observed_levels_distance(fpoe_a,fpoe_g, lambda u,v:wasserstein_2d(np.array([u]).transpose(),np.array([v]).transpose()))
# orig=observed_levels_distance(fpoe_a,fpoe_g)
# mine.loc[('0000','0000')]/orig.loc[('0000','0000')]

#neos_a['list']
# neos_a['mean'].applymap(lambda x:[x])
#pd.DataFrame([a,b],index=['a','b']).fillna([0])
#pd.DataFrame(a.array)
#np.array(neos_a['list'].loc[('0000','0001')].apply(lambda x:[np.mean(x)]))
#neos_a['mean'].loc[('0000','0001')].apply(lambda x:[x]).to_list()
#by_feature(neos_g)['list']
#fpoe_a['mean'].loc[('0000','0000')].sum()
# np.array(neos_a['list'].loc[('0000','0001')].to_list()).transpose().shape[0]

array([[1.41421356, 1.        , 1.41421356, ..., 1.        , 1.        ,
        1.        ],
       [2.        , 3.        , 2.        , ..., 3.        , 3.        ,
        3.        ],
       [1.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [1.41421356, 1.        , 1.41421356, ..., 1.        , 1.        ,
        1.        ],
       [1.41421356, 1.        , 1.41421356, ..., 1.        , 1.        ,
        1.        ]])

array([[0.        , 1.        , 1.        , ..., 1.        , 0.        ,
        0.        ],
       [8.30662386, 8.24621125, 8.24621125, ..., 8.24621125, 8.30662386,
        8.30662386],
       [0.        , 1.        , 1.        , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 0.        , ..., 0.        , 1.        ,
        1.        ],
       [0.        , 1.        , 1.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 1.        , 1.        , ..., 1.        , 0.        ,
        0.        ]])

array([[ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [10.        , 10.        , 10.        , ..., 10.        ,
        10.        , 10.        ],
       [ 8.36660027,  8.36660027,  8.36660027, ...,  8.36660027,
         8.36660027,  8.36660027],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 5.91607978,  5.91607978,  5.91607978, ...,  5.91607978,
         5.91607978,  5.91607978],
       [ 1.41421356,  1.41421356,  1.41421356, ...,  1.41421356,
         1.41421356,  1.41421356]])

array([[48.52834223, 48.64154603, 48.64154603, 48.52834223, 48.31148932,
        48.64154603, 48.64154603, 48.64154603, 48.52834223, 48.64154603,
        48.64154603, 48.52834223, 48.64154603, 48.64154603, 48.52834223,
        48.64154603, 48.64154603, 48.52834223, 48.64154603, 48.52834223,
        48.52834223, 48.64154603, 48.52834223, 48.64154603, 48.64154603,
        48.64154603, 48.64154603, 48.64154603, 48.64154603, 48.64154603,
        48.52834223, 48.43552415, 48.64154603, 48.52834223, 48.52834223,
        48.64154603, 48.64154603, 48.43552415, 48.64154603, 48.64154603,
        48.52834223, 48.43552415, 48.52834223, 48.64154603, 48.02082881,
        48.64154603, 48.64154603, 47.93745926, 48.43552415, 48.64154603,
        48.64154603, 48.64154603, 48.52834223, 48.64154603, 48.64154603,
        48.52834223, 48.64154603, 48.43552415, 48.52834223, 48.52834223,
        48.52834223, 48.64154603, 48.64154603, 48.52834223, 48.31148932,
        48.43552415, 48.64154603, 48.52834223, 48.4

array([[1.        , 1.41421356, 1.        , ..., 1.        , 1.        ,
        1.41421356],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [2.23606798, 1.41421356, 2.23606798, ..., 2.23606798, 2.23606798,
        1.41421356],
       ...,
       [1.        , 1.41421356, 1.        , ..., 1.        , 1.        ,
        1.41421356],
       [3.        , 2.        , 3.        , ..., 3.        , 3.        ,
        2.        ],
       [1.        , 0.        , 1.        , ..., 1.        , 1.        ,
        0.        ]])

...

array([[ 3.        ,  3.60555128,  4.24264069,  3.        ,  3.60555128,
         3.60555128,  3.60555128,  3.        ,  3.16227766,  5.        ,
         3.60555128,  3.16227766,  3.16227766,  3.16227766,  3.16227766,
         3.16227766,  3.16227766,  3.        ,  3.16227766,  3.        ,
         3.16227766,  5.        ,  3.        ,  3.74165739,  3.16227766,
         3.        ,  3.        ,  3.16227766,  3.        ,  3.60555128,
         4.89897949,  3.        ,  3.16227766,  3.        ,  3.60555128,
         4.35889894,  3.60555128,  3.        ,  3.60555128,  3.        ,
         3.16227766,  3.16227766,  3.        ,  3.        ,  3.16227766,
         3.        ,  3.60555128,  3.60555128,  3.60555128,  3.60555128,
         3.16227766,  3.60555128,  3.16227766,  3.60555128,  3.16227766,
         4.24264069,  3.        ,  3.60555128,  3.        ,  3.16227766],
       [ 1.        ,  2.23606798,  3.16227766,  1.        ,  2.23606798,
         2.23606798,  2.23606798,  1.        ,  1.

array([[1.        , 0.        , 1.        , ..., 0.        , 0.        ,
        3.        ],
       [0.        , 1.        , 0.        , ..., 1.        , 1.        ,
        4.        ],
       [1.73205081, 1.41421356, 1.73205081, ..., 1.41421356, 1.41421356,
        3.31662479],
       ...,
       [9.43398113, 9.2736185 , 9.43398113, ..., 9.2736185 , 9.2736185 ,
        9.43398113],
       [1.        , 0.        , 1.        , ..., 0.        , 0.        ,
        3.        ],
       [0.        , 1.        , 0.        , ..., 1.        , 1.        ,
        4.        ]])

array([[1.        , 1.41421356, 1.        , ..., 1.        , 1.41421356,
        1.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [4.47213595, 4.12310563, 4.47213595, ..., 4.47213595, 4.12310563,
        4.47213595],
       [1.        , 1.41421356, 1.        , ..., 1.        , 1.41421356,
        1.        ],
       [1.41421356, 1.        , 1.41421356, ..., 1.41421356, 1.        ,
        1.41421356]])

array([[2.82842712, 2.        , 2.        , ..., 2.        , 2.23606798,
        2.        ],
       [0.        , 2.        , 2.        , ..., 2.        , 1.        ,
        2.        ],
       [9.64365076, 8.06225775, 8.06225775, ..., 8.06225775, 8.83176087,
        8.06225775],
       ...,
       [3.60555128, 3.        , 3.        , ..., 3.        , 3.16227766,
        3.        ],
       [1.        , 2.23606798, 2.23606798, ..., 2.23606798, 1.41421356,
        2.23606798],
       [5.38516481, 5.        , 5.        , ..., 5.        , 5.09901951,
        5.        ]])

author_feature  tweet_feature
0000            0000             391.523081
                0001              48.106746
                0010             138.775467
                0011              90.748523
                0100             186.782432
                                    ...    
1101            1000             308.377773
                1001              35.087863
                1010              48.775291
                1100             174.444979
                1110              21.164414
Length: 131, dtype: float64

In [244]:
mine.loc[('0000','0010')]/orig.loc[('0000','0010')]


90.46178343949046

In [240]:
len(fpoe_g.loc[('0000','0000')]['list'][0])

1796

In [245]:
orig

author_feature  tweet_feature
0000            0000             0.290439
                0001             0.453620
                0010             0.431121
                0011             0.203445
                0100             0.498565
                                   ...   
1101            1000             0.758389
                1001             0.318748
                1010             0.369030
                1100             0.303179
                1110             0.273639
Length: 131, dtype: float64

In [246]:
mine

author_feature  tweet_feature
0000            0000             86.0
                0001             25.0
                0010             39.0
                0011             15.0
                0100             57.0
                                 ... 
1101            1000             93.0
                1001             14.0
                1010              0.0
                1100             24.0
                1110              5.0
Length: 131, dtype: float64

In [269]:
from scipy.stats import wasserstein_distance

# np.random.seed(0)
n = 100

Y1 = np.random.randn(n)
Y2 = np.random.randn(n-10) - 2
d =  np.abs(Y1 - Y2.reshape((n-10, 1)))

assignment = linear_sum_assignment(d)
print(d[assignment].sum()/90)       # 1.9777950447866477
print(wasserstein_distance(Y1, Y2))  #
#np.abs(Y1 - Y2.reshape((n, 1)))

1.7175756547887042
1.888756845699202


In [5]:
s=ast.literal_eval(open('hawk-out/neos/solutions-neos-1338421.hawk-pbs5.pyon').read())

In [10]:
f0=pd.DataFrame([{**d, 'val':v} for v,d in s[('0000','0000')]])

In [16]:
f0.sort_values('discount_factor')

,discount_factor,corr,val
69,0.900000,0.004300,1.406988
66,0.900000,0.004500,1.880012
71,0.910000,0.004200,3.227481
67,0.910000,0.004300,1.148491
31,0.910000,0.004500,1.077906
...,...,...,...
57,0.996094,0.004300,1.138493
42,0.996094,0.003700,1.384988
11,1.000000,0.004648,0.478144
62,1.000000,0.004700,2.301293


In [3]:
g=sp.sparse.load_npz('../kpm/pokec_full.npz')

In [9]:
g[0:990908//2, 0:990908//2]

<495454x495454 sparse matrix of type '<class 'numpy.float64'>'
	with 20002868 stored elements in Compressed Sparse Row format>